In [2]:
#Current imports
import pandas as pd

##Tensorflow
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense


In [ ]:
def doc2vec_model():
    ##Sequential right???
    model = Sequential()

    #Embedding layer
    #GlobalAveragePooling1D layer
    #Flattening

    #Dense layers
    #Optimization such as dropout??


    #binary or multi-class compilation
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [3]:
##Preprocessing libraries
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
##Preprocessing code
##remove stop words?
##lowercase or standardize the format or each word?

##tokenize
tokenizer = Tokenizer()

In [ ]:
##Train actual model


model = doc2vec_model()
##model.fit(, epochs=20, batch_size=10, verbose=1)